# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [22]:
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
data = pd.read_csv("./clean_city_data.csv")
data.head()

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
# Store 'Lat' and 'Lng' into  locations 
locations = data[["Lat", "Lng"]].astype(float)


humidity = data["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
data.columns

Index(['Unnamed: 0', 'City_ID', 'City', 'Cloudiness', 'Country', 'Date',
       'Humidity', 'Lat', 'Lng', 'Max Temp', 'Wind Speed'],
      dtype='object')

In [36]:
data[(data['Max Temp'] < 80) & (data['Max Temp'] > 70) & (data['Wind Speed'] < 10) & (data['Cloudiness'] == 0)]


,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
137,138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
138,139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
175,176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
296,297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
443,444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [37]:
filtered_data_indxs = data[(data['Max Temp'] < 80) & (data['Max Temp'] > 70) & (data['Wind Speed'] < 10) & (data['Cloudiness'] == 0)].index
filtered_data_indxs

Int64Index([7, 70, 88, 137, 138, 175, 296, 443], dtype='int64')

In [44]:
clean_data = data.drop(data.index[filtered_data_indxs], inplace=False)
clean_data.head()

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [95]:
target_search = "Hotels"
#target_city = "Irvine, California"
irvine_coords = "33.6846, -117.8265"
radius = 5000
params = {
    "location": irvine_coords,
    "types": target_search,
    "radius": radius,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


print("Hotels Near Irvine")


hotels_Irvine = requests.get(base_url, params).json()
print(f"Name: {hotels_Irvine['results'][1]['name']}")
print(f"City:{hotels_Irvine['results'][1]['plus_code']['compound_code']}")
print(f"Lat/Lng:{hotels_Irvine['results'][1]['geometry']['location']}")

Hotels Near Irvine
Name: Residence Inn by Marriott Irvine John Wayne Airport/Orange County
City:M5J7+76 Irvine, California, United States
Lat/Lng:{'lat': 33.680649, 'lng': -117.836971}


In [91]:
hotels_Irvine

{'html_attributions': [],
 'next_page_token': 'CqQCGwEAAIVdHRRuqw2vC9U8nOlDBU-DToxDZaBnnFmwWjO_G2pLgNA2xZlxcQpF3RN8pBZN2r3z990WwII1ImieLaWZi7XgQz7PNLX7_ArIQDBI8tvyIymPPOl5EezlueEERVyl9ReLDXi0h37sFOo_pZqtSjDtON57hk7kjz81Ii7VrisNulA-e_ofzEDzAXMhnOSG6lNwPtKAmrFnLPhn7b4gcOsZNQEvEtB_aBzdAGGdKJat7bj_YO0izWn4zJan64vIrAa5nauXcCj7iBgXRO9_49oB8jYlV1XK9na2l67wHpV_WXnf4tR3hJyEDoCoHgje14dd-BF72cNLdosU1Qz2rdcH11F_kN6I4_BG0WmpCcWjJD4YeN3tFv7ZDoovnLps-BIQjYzK_D_9YHVN2-jurSIN4BoUeQGFqdnzvhSLx-6YmphhYpu5ru0',
 'results': [{'geometry': {'location': {'lat': 33.6845673,
     'lng': -117.8265049},
    'viewport': {'northeast': {'lat': 33.7736521, 'lng': -117.6780258},
     'southwest': {'lat': 33.5993914, 'lng': -117.8687519}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png',
   'id': '8bcf62cf410aa064673d19f76655e474040960ec',
   'name': 'Irvine',
   'photos': [{'height': 3024,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116418254005302142023">Wi

In [101]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

frame_dict = {
    'Hotel Name': [hotels_Irvine['results'][1]['name'], 
                   hotels_Irvine['results'][2]['name']],
    'City': [hotels_Irvine['results'][1]['plus_code']['compound_code'], 
             hotels_Irvine['results'][2]['plus_code']['compound_code']],
    'Country': ['United States', 'United States'],
    'Lat': [hotels_Irvine['results'][1]['geometry']['location']['lat'], 
            hotels_Irvine['results'][2]['geometry']['location']['lat']],
    'Lng': [hotels_Irvine['results'][1]['geometry']['location']['lng'],
           hotels_Irvine['results'][2]['geometry']['location']['lng']]
}
# Store the DataFrame Row
hotel_df = pd.DataFrame.from_dict(frame_dict)

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [103]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))